# 多層感知器Regularization實作 模型載入


## 載入套件

In [1]:
import tensorflow as tf
import input_data
tf.set_random_seed(1234)

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 訓練資料

In [2]:
batch_xs, batch_ys = mnist.train.next_batch(5)

## 建立Computational Graph

In [3]:
x_ = tf.placeholder(tf.float32, [None, 784], name="x_")
y_ = tf.placeholder(tf.float32, [None, 10], name="y_")

W1 = tf.Variable(tf.truncated_normal([784,800], stddev=0.1), name="W1")
b1 = tf.Variable(tf.zeros([800]), name="b1")
h1 = tf.nn.sigmoid(tf.matmul(x_, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([800,10], stddev=0.1), name="W2")
b2 = tf.Variable(tf.zeros([10]), name="b2")
y = tf.nn.softmax(tf.matmul(h1, W2) + b2)

cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

lambda_ = tf.placeholder(tf.float32, name="lambda")
regularizer = tf.reduce_sum(tf.square(W1))+tf.reduce_sum(tf.square(W2))

cost = cross_entropy + lambda_*regularizer


optimizer = tf.train.GradientDescentOptimizer(0.003)
trainer = optimizer.minimize(cost)

init = tf.initialize_all_variables()
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Session

In [4]:
sess = tf.Session()
sess.run(init)

## 建立成效評估用的Computational Graph

In [5]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Saver

In [6]:
saver = tf.train.Saver(max_to_keep=10)


## 模型載入＆模型選擇

In [7]:
lambdas = [0,0.00001,0.0001,0.001,0.01,0.1]

for lamb in lambdas:
    model_name = "model_regularization_%s.ckpt"%( str(lamb).replace(".","") )
    saver.restore(sess, model_name)
    
    valid_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.validation.images, y_: mnist.validation.labels, lambda_:lamb})
    test_accurarcy = sess.run(accuracy, feed_dict={x_: mnist.test.images, y_: mnist.test.labels, lambda_:lamb})
    print "lambda:%s valid:%s, test:%s"%(lamb, valid_accurarcy,test_accurarcy)

lambda:0 valid:0.9754, test:0.9731
lambda:1e-05 valid:0.975, test:0.9717
lambda:0.0001 valid:0.9774, test:0.9736
lambda:0.001 valid:0.976, test:0.9722
lambda:0.01 valid:0.9724, test:0.9683
lambda:0.1 valid:0.9204, test:0.9108
